# 1. Exploration

In [1]:
import json
from queue import PriorityQueue
import math

f1 = open('../data/G.json')
f2 = open('../data/Coord.json')
f3 = open('../data/Dist.json')
f4 = open('../data/Cost.json')
MAX_BUDGET = 287932

In [2]:
from collections import defaultdict
G = json.load(f1)
# h(n)
Coord = json.load(f2)
# g(n)
Dist = json.load(f3)
Cost = defaultdict(int, json.load(f4))

In [3]:
def calc_hn(cur_node, target):
    x1, y1 = Coord[cur_node]
    x2, y2 = Coord[target]
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

In [4]:
def get_graph(fp='../data/G.json'):
    f = open(fp)
    return json.load(f)

In [5]:
def path_dist(path):
    total_dist = 0
    for i in range(1, len(path)):
        total_dist += Dist[f'{path[i-1]},{path[i]}']
    return total_dist

In [6]:
def path_cost(path):
    total_cost = 0
    for i in range(1, len(path)):
        total_cost += Cost[f'{path[i-1]},{path[i]}']
    return total_cost

In [7]:
def A_star(graph, source, target):
    # tracks the A* score of visited nodes
    visited = {}
    # priority queue of score, path. Lower scores are higher on priority.
    queue = PriorityQueue()
    queue.put((0, [source], 0))
    while not queue.empty():
        # get the highest priority path
        candidate = queue.get()
        cur_node = candidate[1][-1]
        
        visited[cur_node] = candidate[0]
        if cur_node == target:
            return candidate
        # add all the edges to the priority queue
        for node in graph[cur_node]:
            # if node has been visited, only skip if its recorded A* score is <= current calculated score
            if node in visited and visited[node] <= candidate[0]+Dist[f'{cur_node},{node}']+calc_hn(node, target):
                continue
            # create a new path with the node from the edge
            new_path = list(candidate[1]) + [node]
            # add distance and new path to queue
            queue.put((candidate[0] + Dist[f'{cur_node},{node}'] + calc_hn(node,target), 
                       new_path, 
                       candidate[2]+Cost[f'{cur_node},{node}']))
    return None, None, None

In [8]:
_, path, _ = A_star(G, '1', '50')

In [9]:
print('Shortest path:', '->'.join(node for node in path) + '.')
print(f'Shortest distance: {path_dist(path)}.')
print(f'Total energy cost: {path_cost(path)}')

Shortest path: 1->1363->1358->1355->1274->1143->1264->1239->1237->1236->1228->1227->941->936->935->923->930->919->927->925->924->906->901->755->677->663->660->659->366->365->364->349->347->345->342->320->343->297->294->295->196->202->181->182->184->215->216->217->96->98->221->232->222->237->236->4814->4808->4806->4804->4787->4788->4789->5149->5150->5180->5195->5196->5197->5199->5166->5165->5163->5162->5169->5170->5172->5174->5273->5266->5265->5264->5255->5267->5269->5257->5268->50.
Shortest distance: 224198.03681434665.
Total energy cost: 452855


In [12]:
from tqdm import tqdm

In [10]:
def A_star(graph, source, target):
    # tracks the A* score of visited nodes
    visited = {}
    # priority queue of score, path. Lower scores are higher on priority.
    queue = PriorityQueue()
    queue.put((0, [source], 0))
    while not queue.empty():
        # get the highest priority path
        candidate = queue.get()
        cur_node = candidate[1][-1]
        
        visited[cur_node] = candidate[0]
        if cur_node == target:
            return candidate
        # add all the edges to the priority queue
        for node in graph[cur_node]:
            # if node has been visited, only skip if its recorded A* score is <= current calculated score
            if node in visited and visited[node] <= candidate[0]+Dist[f'{cur_node},{node}']+calc_hn(node, target):
                continue
            # create a new path with the node from the edge
            new_path = list(candidate[1]) + [node]
            # add distance and new path to queue
            queue.put((candidate[0] + Dist[f'{cur_node},{node}'] + calc_hn(node,target), 
                       new_path, 
                       candidate[2]+Cost[f'{cur_node},{node}']))
    return None, None, None

def yens_ksp(graph, func, source, target, budget, K=5, A=None, terminate=False):
    if not A:
        A = [func(graph, source, target)[1]]
    B = PriorityQueue() #Queue of candidate paths

    for k in range(1, K):
        for i in range(len(A[k-1]) - 1):
            G = get_graph() #New graph for each iteration
            spur_node = A[k-1][i]
            root_path = A[k-1][:i]
            for path in A:
                if len(path) - 1 > i and root_path == path[:i]:
                    # remove edge connecting root path to spur node from Graph, if edge isn't
                    # already deleted
                    if path[i+1] in G[path[i]]:
                        G[path[i]].remove(path[i+1])
            _, spur_path, _ = func(G, spur_node, target)
            # if there exists a path from spur node to target
            if spur_path:
                found_in_B = False
                for paths in B.queue:
                    # if path exists then dont add total_path info in.
                    if paths[1]==spur_path:
                        found_in_B = True
                        break
                # path is new to B, add it in
                if not found_in_B:
                    total_path = root_path + spur_path
                    B.put((path_dist(total_path), total_path, path_cost(total_path)))
        # since B is already sorted, we first try to find whether shortest paths in B
        # satisfy our budget constraint. If not, find the shortest path and add to A.
        if terminate:
            return A[0]
        while B:
            dist, path, cost = B.get()
            # this is a k-th version of shortest path
            if path not in A:
                if cost <= budget:
                    return path, dist, cost
                A.append(path)
    return A, None, None

In [ ]:
new_path, new_dist, new_cost = yens_ksp(G, A_star, '1', '50', MAX_BUDGET, K=2)

In [ ]:
print('Shortest path:', '->'.join(node for node in new_path) + '.')
print(f'Shortest distance: {path_dist(path)}.')
print(f'Total energy cost: {path_cost(path)}')